<a href="https://colab.research.google.com/github/Naveenjeyasankar/AI-PROJECTS/blob/main/VITRES50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['TP_CPP_MIN_LOG_LEVEL']="2"
import tensorflow as tf
from tensorflow.keras.layers import*
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50





In [ ]:
def transformer_encoder(x,cf):
  skip_2=x
  x=LayerNormalization()(x)
  x=MultiHeadAttention(
      num_heads=cf["num_heads"],key_dim=cf["hidden_dim"]

  )(x,x)
  x=Add()([x,skip_2])
  return x

In [ ]:
class ClassToken(Layer):
  def __init__(self,):
    super().__init__()

    def build(self,input_shape):
      w_init=tf.random.normal.initalizer()
      self.w=tf.Variable(
      initiial_vale=w_init(shape=(1,1,input_shape[-1]),dtype="float32"),
      trainable=True
      )
      def call(self,inputs):
        batch_size=tf.shape(inputs)[0]
        hidden_dim=self.w.shape[-1]


        cls=tf.broadcast_to(self.w, [batch_size,1,hidden_dim ])
        cls=tf.cast(cls,dtype=inputs.dtype)
        return cls

In [ ]:
from keras.api._v2.keras import layers
from numpy.core.multiarray import concatenate
from typing_extensions import Concatenate
def ResNet50vit(cf):
  inputs=Input((cf["image_size"],cf["image_size"],cf["num_channels"]))

  resnet50=ResNet50(include_top=False,weights="imagenet",input_tensor=inputs)
  output=resnet50.output


  patch_embed= Conv2D(cf["hidden_dim"],
                     kernel_size=cf["patch_size"],
                     padding="same")(output)

  _,h,w,f=patch_embed.shape
  patch_embed=Reshape((h*w,f))(patch_embed)

  position=tf.range(start=0,limit=cf["num_patches"],delta=1)


  pos_embed=Embedding(input_dim=cf["num_patches"],output_dim = cf["hidden_dim"])(position)




  embed=patch_embed+pos_embed

  token= ClassToken()(embed)
  x=layers.Concatenate()([token,embed])


  for _ in range(cf["num_layers"]):
     x =transformer_encoder(x,cf)


  x=LayerNormalization()(x)
  x=x[: ,0,:]
  x=Dense(cf["num_classes"],activation ="softmax")(x)

  model=Model(inputs,x)

  return model

if __name__ == "__main__":
  config={}
  config["num_layers"]=12
  config["hidden_dim"]=8
  config["mlp_dim"]=512
  config["num_heads"]=12
  config["dropout_rate"]=0.5

  config["image_size"]=512
  config["patch_size"]=32

  config["num_patches"]=int(config["image_size"]**2/config["patch_size"]**2)
  config["num_channels"]=3
  config["num_classes"]=1

  model=ResNet50vit(config)
  model.summary()





Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_17 (InputLayer)       [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 518, 518, 3)          0         ['input_17[0][0]']            
                                                                                                  
 conv1_conv (Conv2D)         (None, 256, 256, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 256, 256, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        